<a href="https://colab.research.google.com/github/Blusooyeon/Studying_for_Bigdata_Test/blob/main/T2_airline_satisfying_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####test 데이터에 대해서 neutral or dissatisfied라고 예측할 확률을 구하고 그 확률 값을 제출하라

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,id
0,0,Female,Loyal Customer,54,Personal Travel,Eco,1068,3,4,3,...,5,5,3,5,3,5,3,47,22.0,NaN
1,2,Male,Loyal Customer,20,Personal Travel,Eco,1546,4,4,4,...,4,3,3,4,4,4,4,5,2.0,NaN
2,3,Male,Loyal Customer,59,Business travel,Business,2962,0,4,0,...,1,1,1,1,5,1,4,54,46.0,NaN
3,4,Male,Loyal Customer,35,Business travel,Eco Plus,106,5,4,4,...,5,2,1,5,4,4,5,130,121.0,NaN
4,5,Female,Loyal Customer,9,Business travel,Business,2917,3,3,3,...,4,4,4,5,4,3,4,0,0.0,NaN


,ID,satisfaction
0,0,neutral or dissatisfied
1,2,neutral or dissatisfied
2,3,satisfied
3,4,satisfied
4,5,satisfied


In [2]:
x_train.isnull().sum()

ID                                       0
Gender                                   0
Customer Type                            0
Age                                      0
Type of Travel                           0
Class                                    0
Flight Distance                          0
Inflight wifi service                    0
Departure/Arrival time convenient        0
Ease of Online booking                   0
Gate location                            0
Food and drink                           0
Online boarding                          0
Seat comfort                             0
Inflight entertainment                   0
On-board service                         0
Leg room service                         0
Baggage handling                         0
Checkin service                          0
Inflight service                         0
Cleanliness                              0
Departure Delay in Minutes               0
Arrival Delay in Minutes               256
id         

In [3]:
mean_values = x_train['Arrival Delay in Minutes'].mean()
x_train['Arrival Delay in Minutes'] = x_train['Arrival Delay in Minutes'].fillna(mean_values)

# data leakage 때문에 결측치는 train값으로 채우는게 원칙이나 신경쓰기 어렵다면 test의 결측치는 test의 평균값으로 대치하세요
x_test['Arrival Delay in Minutes'] = x_test['Arrival Delay in Minutes'].fillna(mean_values)

In [4]:
drop_col = ['ID','id']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['satisfaction']


x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

In [6]:
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train_dummies, y, test_size=0.33, random_state=42,stratify=y)
rf = RandomForestClassifier(random_state =23)
rf.fit(X_train,Y_train)


RandomForestClassifier(random_state=23)

In [7]:

from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score

#model_score
predict_train_label = rf.predict(X_train)
predict_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

In [8]:
print('train accuracy :', accuracy_score(Y_train,predict_train_label))
print('validation accuracy :', accuracy_score(Y_validation,predict_validation_label))
print('\n')
print('train f1_score :', f1_score(Y_train,predict_train_label ,pos_label = 'neutral or dissatisfied'))
print('validation f1_score :', f1_score(Y_validation,predict_validation_label ,pos_label = 'neutral or dissatisfied'))
print('\n')
print('train recall_score :', recall_score(Y_train,predict_train_label,pos_label = 'neutral or dissatisfied'))
print('validation recall_score :', recall_score(Y_validation,predict_validation_label,pos_label = 'neutral or dissatisfied'))
print('\n')
print('train precision_score :', precision_score(Y_train,predict_train_label,pos_label = 'neutral or dissatisfied'))
print('validation precision_score :', precision_score(Y_validation,predict_validation_label,pos_label = 'neutral or dissatisfied'))
print('\n')
print('train auc :', roc_auc_score(Y_train,predict_train_proba))
print('validation auc :', roc_auc_score(Y_validation,predict_validation_prob))

train accuracy : 1.0
validation accuracy : 0.9611388574969925


train f1_score : 1.0
validation f1_score : 0.9661221636051611


train recall_score : 1.0
validation recall_score : 0.9778692743180648


train precision_score : 1.0
validation precision_score : 0.954653937947494


train auc : 1.0
validation auc : 0.9936992375795042


In [11]:
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,0]

array([0.82, 0.97, 1.  , ..., 0.  , 0.99, 0.88])